<a href="https://colab.research.google.com/github/kidgu/VideoCategorization/blob/main/Rulebase_word_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import time
import pickle

In [ ]:
class TrieNode:
    """A node in the trie structure"""

    def __init__(self, char):
        # the character stored in this node
        self.char = char

        # whether this can be the end of a word
        self.is_end = False

        # a counter indicating how many times a word is inserted
        # (if this node's is_end is True)
        self.counter = 0

        # a dictionary of child nodes
        # keys are characters, values are nodes
        self.children = {}

In [ ]:
class Trie(object):
    """The trie object"""

    def __init__(self):
        """
        The trie has at least the root node.
        The root node does not store any character
        """
        self.root = TrieNode("")

    def insert(self, word):
        """Insert a word into the trie"""
        node = self.root

        # Loop through each character in the word
        # Check if there is no child containing the character, create a new child for the current node
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                # If a character is not found,
                # create a new node in the trie
                new_node = TrieNode(char)
                node.children[char] = new_node
                node = new_node

        # Mark the end of a word
        node.is_end = True

        # Increment the counter to indicate that we see this word once more
        node.counter += 1

    def dfs(self, node, prefix):
        """Depth-first traversal of the trie

        Args:
            - node: the node to start with
            - prefix: the current prefix, for tracing a
                word while traversing the trie
        """
        # self.output = []
        if node.is_end:
            self.output.append((prefix + node.char, node.counter))

        for child in node.children.values():
            self.dfs(child, prefix + node.char)

    def query(self, x):
        """Given an input (a prefix), retrieve all words stored in
        the trie with that prefix, sort the words by the number of
        times they have been inserted
        """
        # Use a variable within the class to keep all possible outputs
        # As there can be more than one word with such prefix
        self.output = []
        node = self.root

        # Check if the prefix is in the trie
        for char in x:
            if char in node.children:
                node = node.children[char]
            else:
                # cannot found the prefix, return empty list
                return []

        # Traverse the trie to get all candidates
        self.dfs(node, x[:-1])

        # Sort the results in reverse order and return --> dont need to sort
        # return sorted(self.output, key=lambda x: x[1], reverse=True)
        return self.output

    def __repr__(self):
        def recur(node, indent):
            return "".join(indent + key + ("$" if child.is_end else "")
                                  + recur(child, indent + " ")
                for key, child in node.children.items())

        return recur(self.root, "\n")

In [ ]:
# Program to read all the lines as a list in a file
# using readlines() function
file = open("/content/bad_words.txt", "r", encoding="utf-8")
content=file.readlines()
# print(content)
file.close()

In [ ]:
print(type(content))
print(len(content))

<class 'list'>
1


In [ ]:
split_words = content[0].split(",")
# decode to utf-8, and strip space both side of string
bad_words = [ word.encode().decode("utf-8").strip() for word in split_words]
print(bad_words[:5])
print(len(bad_words))

['a cmnl', 'a mầy', 'a shit', 'a shitty', 'a sướng']
3738


In [ ]:
import random
group_of_items = bad_words  # a sequence or set will work here.
num_to_select = 500                           # set the number to select here.
list_of_random_items = random.sample(group_of_items, num_to_select)

In [ ]:
print(list_of_random_items[:5])
bad_words = list_of_random_items

['chõ miệng', 'gái dâm', 'dị hơm', 'sextop1.net', 'đéo tưởng']


In [ ]:
bad_words_trie = Trie()

In [ ]:
for word in bad_words:
    bad_words_trie.insert(word)

In [ ]:
# print(bad_words_trie)

In [ ]:
bad_words_trie.query("chó")

[('chó', 1)]

In [ ]:
# https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict-or-any-other-python-object/33245595#33245595
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

save_object(bad_words_trie, 'bad_words_trie.pkl')

In [ ]:
with open('bad_words_trie.pkl', 'rb') as inp:
    bad_words_trie = pickle.load(inp)
    print(bad_words_trie.query("chó"))

[('chó', 1)]


In [ ]:

text = '''
1. Nếu ai đó nói xấu sau lưng bạn Thì hãy tát vào mặt nó và nói rằng “Tao không những xấu mà còn ác nữa baby à!”2. Lưỡi dao lam, vừa mỏng vừa manh Miệng nhiều người, vừa bẩn vừa tanh.(adsbygoogle = window.adsbygoogle || []).push({});3. Đừng bao giờ chửi ai đó là CON CHÓ Bởi như thế chẳng khác nào bạn đang khen họ sống có tình có nghĩa.4. Nhiều khi tớ muốn được sống như bạn Sinh ra đã có bản tính khốn nạn trong người.5. Trước mặt thì chia sẽ, sau lưng thì chia rẽ Xin lỗi hai chữ TÌNH BẠN, bạn không xứng đáng đứng cùng tôi.6. Lưu manh giả danh trí thức Khốn nạn mà giả dạng dễ thương.7. Đừng lãng phí những ngày đẹp trời Chỉ để quan tâm những lời thiên hạ.8. Đời thì hối hả, bon chen Con người thì hồn nhiên giả tạo.9. Nếu ai đó nói xấu bạn thì hãy mặc kệ. Hãy sống sao cho người ta thấy bạn sống tốt hơn gấp vạn lần so với những gì họ nói.10. Những kẻ chỉ biết nói xấu sau lưng, đó chỉ là những kẻ chỉ có thể đứng mãi đằng sau lưng chúng ta mà thôi. Đời luôn nhìn về phía trước, chứ đâu ai muốn nhìn lại phía sau mãi.11. Nếu không tử tế thì đừng đẹp trai.12. Người ta đã cố gắng nói dối Thì mình ngại gì mà không giả vờ tin.13. Luật đời dạy em cách giả tạo Em quật lại đời bằng bản chất của riêng em.14. Tớ không phải là nai, tớ là cáo Có khi tớ còn bố láo hơn cả hồ ly.15. Nếu chỉ nhìn bề ngoài mà đoán được một người Thì trên đời này đã không còn tồn tại hai chữ KHÔNG NGỜ.16. Sống ở đời bạn phải biết cái khôn nằm đâu chứ đừng tỏ vẻ, trong não của bạn hình như không được xoắn lắm.17. Ghen ăn tức ở muôn đời nát Bởi vì nên không ngốc đầu lên nổi.18. Thà khốn nạn công khai Còn hơn giả nai thánh thiện.19. Đừng có xấu mà bày đặt hư cấu Nói nhảm mà còn không biết mặc cảm nữa.20. Khi bạn chỉ biết cách đứng đằng sau và chỉ trỏ này kia, thì có nghĩa là bạn đang thấp kém hơn người ta rất nhiều đó bạn.
'''

# lower case
text = text.lower()
# remove special character:
text = re.sub(r'•|!|#|\s|#|\\|\$|%|&|\^|\>|\<|\~|\+|\-|\_|\/|\*|\}|\{|\]|\[|\?|\|', ' ', text)
# split by mulitiple delimilers
words = re.split(';|,|\.| ', text)
# convert list to set to reduce duplicate words:
set_words = set(words)
print(set_words)

{'', 'lần', 'để', 'cái', 'nói', 'mặc', 'push(', 'hạ', 'chẳng', 'láo', 'nên', '5', 'ly', 'dễ', 'khác', 'ngờ', 'cảm', 'trời', 'đằng', 'tanh', 'họ', 'sống', 'này', 'những', 'chửi', 'thánh', 'không', 'vờ', 'đó', 'tính', 'trong', 'chỉ', 'mình', 'của', 'riêng', 'hồn', 'xin', 'hai', 'hình', '15', 'trí', 'là', 'hả', 'thấp', 'tại', '(adsbygoogle', 'thấy', 'như', 'bản', 'thương', 'nếu', 'luật', 'nai', 'nổi', 'con', 'lưu', 'gắng', 'thiện', 'có', 'muôn', 'tâm', 'trước', 'đáng', 'quan', 'em', '8', 'hãy', 'bày', '7', 'giả', 'phí', 'cho', 'cách', 'được', 'chia', 'hư', 'cáo', 'nhiên', 'mãi', 'nát', 'tin', 'bởi', 'bao', 'bon', 'vào', 'khai', 'window', 'ra', 'một', 'tạo', 'mỏng', 'ta', 'thôi', 'lên', '13', 'còn', 'vì', 'lam', 'đẹp', 'đâu', 'đầu', 'rằng', 'tốt', 'vạn', 'nhiều', 'khen', 'dao', 'dối', 'tát', 'miệng', 'gấp', 'thể', 'tỏ', 'đã', 'lời', 'ác', 'ngày', 'trên', 'dạng', '=', 'thì', 'à', 'nhảm', '10', '4', '“tao', 'bẩn', 'thế', 'lưỡi', 'rẽ', 'đời', 'với', '”2', 'nghĩa', 'tôi', 'cả', '9', 'quật', 'b

In [ ]:

def get_words_from_text(text):
    # return a list of distinct words in a text
    text = text.lower()
    text = re.sub("[^0-9a-zA-ZàáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ@]",        # Anything except 0..9, a..z and A..Z
       " ",                    # replaced with space
        text)
    return set(text.split())

In [ ]:
print(len(get_words_from_text('')))

0


In [ ]:
start = time.time()

possible_bad_words = []

for word in set_words:
    if len(word) != 0:
        # print(word)
        possible_bad_words.extend(bad_words_trie.query(word))
set_posible_bad_words = set([x for (x, y) in possible_bad_words])
# print(set_posible_bad_words)
# print(len(set_posible_bad_words))
for posible_bad_word in set_posible_bad_words:
    if posible_bad_word in text:
        print(posible_bad_word)


end = time.time()
print((end - start)*1000)

# vì có từ "khốn nạn" nên xuất hiện cả "khốn" và "khốn nạn"

bố láo
bày đặt
chó
5.589008331298828


In [ ]:
start = time.time()

for word in words:
    if len(word) != 0:
        # print(word)
        possible_bad_words = bad_words_trie.query(word)
        if len(possible_bad_words) > 0:
            for (possible_bad_word, number) in possible_bad_words:
                if possible_bad_word in text:
                    print(possible_bad_word)


end = time.time()
print((end - start)*1000)

lưỡi
chó
khốn
khốn n
khốn nạ
khốn nạn
lưu manh
khốn
khốn n
khốn nạ
khốn nạn
bố láo
láo
khốn
khốn n
khốn nạ
khốn nạn
bày đặt
nhảm
12.395620346069336


In [ ]:
start = time.time()

for bad_word in bad_words:
    if bad_word in text: # cần check thêm đk từ nằm trong từ
        print(bad_word)

end = time.time()
print((end - start)*1000)

bày đặt
bố láo
chó
khốn
khốn n
khốn nạ
khốn nạn
láo
lưu manh
lưỡi
nhảm
ti
ẳng
ếu

15.518665313720703
